In [174]:
# TODO: use `order_no` var

In [175]:
import pandas as pd
from services import get_engine
import graphviz
import numpy as np
import math

pd.options.display.max_columns = None

In [176]:
engine_unf = get_engine(fname='../credentials/.prod_unf')

Connected to bi@prod_unf.


In [177]:
engine_analytics = get_engine(fname='../credentials/.server_analytics')

Connected to Analytics_root@PROD_ANALYTICS.


In [178]:
order_params_query = '''
SELECT
    CAST(DATEADD([YEAR], -2000, orders._Fld16402) AS date) AS calcDate, 
    price_types.[_Description] AS priceType,
    CAST(price_types.[_IDRRef] AS uniqueidentifier) AS priceTypeId
FROM
    _Document164X1 AS orders
    LEFT JOIN _InfoRg16413 as numbers on numbers._Fld16426RRef = orders._idrref
    LEFT JOIN _Reference36 AS price_types ON price_types._IDRRef = orders._Fld3599RRef
WHERE 
    numbers._Fld16427 LIKE '13925/13/%'
'''

order_params = pd.read_sql(order_params_query, engine_unf)
calc_date = order_params.calcDate.values[0]
price_type_id = order_params.priceTypeId.values[0]
price_type = order_params.priceType.values[0]

In [179]:
order_query = '''
-- earned value report.sql
-- get order
SELECT
    order_tbl._LineNo3639 AS orderRowNo
    , order_tbl._Fld35933 AS stage
    , room._Description AS room
    , tech._Description AS tech
    , spec._Description AS spec
    , CAST(spec._IDRRef AS uniqueidentifier) AS specId
    , item._Description AS item
    , CAST(item._IDRRef AS uniqueidentifier) AS itemId
    , order_tbl._Fld3644 AS qty
    , batch._Description AS batch
    , CAST(batch._IDRRef AS uniqueidentifier) AS batchId
FROM 
    _Document164X1 orders
    LEFT JOIN _InfoRg16413 AS numbers ON numbers._Fld16426RRef = orders._IDRRef
    LEFT JOIN _Document164_VT3638X1 AS order_tbl 
        ON order_tbl._Document164_IDRRef = orders._IDRRef
    LEFT JOIN _Reference76 AS item ON item._IDRRef = order_tbl._Fld3640RRef
    LEFT JOIN _Reference79 as tech ON item._Fld1533RRef = tech._IDRRef
    LEFT JOIN _Reference16143 AS room ON room._IDRRef = order_tbl._Fld16181RRef
    LEFT JOIN _Reference89 AS batch ON batch._IDRRef = order_tbl._Fld3643RRef
    LEFT JOIN _Reference16144 AS spec ON order_tbl._Fld16182RRef = spec._IDRRef
WHERE
    numbers._Fld16427 LIKE '13925/13/%'     -- номерЗП
    ORDER BY
        order_tbl._LineNo3639
'''

order = pd.read_sql(order_query, engine_unf)
order.orderRowNo = order.orderRowNo.astype(int)

# Production Orders

In [180]:
# list production orders based on the customer order
# including deleted and not posted prod orders to find all subordinated documents
# even if they are subordinated to invalidated ones
prods_query_base = '''
-- earned value report.sql
-- get prod orders    
SELECT CAST(prod._IDRRef AS uniqueidentifier) AS prodId,
    prod._Number AS prodNo,
    status._Description AS prodStatus,
    item._Description AS item,
    batch._Description AS batch,
    CAST(batch._IDRRef AS uniqueidentifier) AS batchId,
    tech.[_Description] AS tech,
    owner._Description AS owner,
    CAST(owner._IDRRef AS uniqueidentifier) AS ownerId,
    parent_prod.[_Number] AS parentProdNo,
    CAST(parent_prod.[_IDRRef] AS uniqueidentifier) AS parentProdId,
    parent_item.[_Description] AS parentItem,
    parent_tech.[_Description] AS parentTech,
    parent_batch.[_Description] AS parentBatch,
    CAST(parent_batch._IDRRef AS uniqueidentifier) AS parentBatchId,
    CAST(prod._Marked AS int) AS deleted,
    CAST(prod._Posted AS int) AS posted
FROM _Document163 AS prod
    LEFT JOIN _Reference117 AS status ON prod._Fld3556RRef = status._IDRRef
    LEFT JOIN _Document164X1 AS ordr ON ordr.[_IDRRef] = prod.[_Fld3543RRef]
    LEFT JOIN _InfoRg16413 AS ordr_no ON ordr_no._Fld16426RRef = ordr.[_IDRRef]
    LEFT JOIN _Document163_VT3562 AS prod_tbl ON prod._IDRRef = prod_tbl._Document163_IDRRef
    LEFT JOIN _Reference76 AS item ON item._IDRRef = prod_tbl._Fld3564RRef
    LEFT JOIN _Reference76 AS owner ON owner._IDRRef = prod_tbl._Fld33461RRef
    LEFT JOIN _Reference89 AS batch ON batch._IDRRef = prod_tbl._Fld16259RRef
    LEFT JOIN _Reference79 as tech ON item._Fld1533RRef = tech._IDRRef
    LEFT JOIN _Document163 AS parent_prod ON parent_prod.[_IDRRef] = prod._Fld3555RRef
    LEFT JOIN _Document163_VT3562 AS parent_prod_tbl ON parent_prod._IDRRef = parent_prod_tbl._Document163_IDRRef
    LEFT JOIN _Reference89 AS parent_batch ON parent_batch._IDRRef = parent_prod_tbl._Fld16259RRef
    LEFT JOIN _Reference76 AS parent_item ON parent_item._IDRRef = parent_prod_tbl._Fld3564RRef
    LEFT JOIN _Reference79 as parent_tech ON parent_item._Fld1533RRef = parent_tech._IDRRef
-- need to find all job tickets, not only subordinated to posted prod orders
-- WHERE CAST(prod._Posted AS int) = 1  

'''
prods_query0 = prods_query_base + "WHERE ordr_no._Fld16427 LIKE '13925/13/%'"

prods = pd.read_sql(prods_query0, engine_unf)

In [181]:
# list production orders based on other production orders and not directly based on customer order
def get_all_prods(prods0: pd.DataFrame) -> pd.DataFrame:
    parent_prod_ids = "', '".join(prods0.prodId)
    prods_query1 = prods_query_base + f"""
        -- based on parent prods
        WHERE CAST(parent_prod._IDRRef AS uniqueidentifier) IN ('{parent_prod_ids}')  
        -- not already listed as parent prods. 
        -- TODO: excessive?
        AND CAST(prod._IDRRef AS uniqueidentifier) NOT IN ('{parent_prod_ids}')     
    """
    new_prods = pd.read_sql(prods_query1, engine_unf)
    if len(new_prods) == 0:
        return pd.DataFrame(columns=prods0.columns)

    return pd.concat([new_prods, get_all_prods(new_prods)])

parent_prod_ids = "', '".join(prods.prodId)
prods = pd.concat([prods, get_all_prods(prods)])

In [182]:
prods =prods.merge(
        order[order.batchId.notnull()][['batchId', 'item', 'orderRowNo', 'spec', 'specId']], 
        how='left',
        on=['item', 'batchId'],
        suffixes=('', '_')
)

In [183]:
prods = prods.merge(
        order[order.batchId.notnull()][['batchId', 'item', 'orderRowNo', 'spec', 'specId']], 
        how='left',
        left_on=['parentItem', 'parentBatchId'],
        right_on=['item', 'batchId'],
        suffixes=('', '_parent')
)
prods.drop(columns='batchId_parent', inplace=True)

In [184]:
prods = prods[[
    'prodId',
    'prodNo',
    'prodStatus',
    'deleted',
    'posted',
    'item',
    'batch',
    'batchId',
    'tech',
    'orderRowNo',
    'spec',
    'specId',
    'owner',
    'ownerId',
    'parentProdNo',
    'parentProdId',
    'parentItem',
    'parentTech',
    'parentBatch',
    'parentBatchId',
    'item_parent',
    'orderRowNo_parent',
    'specId_parent',
    'spec_parent'
    ]]

In [185]:
# шаблон не должен подчиняться протяжке
parent_cols = [
    'parentProdNo',
    'parentProdId',
    'parentItem',
    'parentBatch',
    'parentBatchId',
    'item_parent',
    'orderRowNo_parent',
    'spec_parent',
    'parentTech'
]

for col in parent_cols:
    prods[col] = prods.apply(
        lambda r: None if r.tech=='Шаблон' and r.parentTech=='Протяжка'
        else r[col] 
        , axis=1
    )

In [186]:
# верхние ЗНП МФР выводятся из под тиража
for col in parent_cols:
    prods[col] = prods.apply(
        lambda r: 
        None if r.tech in ['Модельные работы', 'Формовочные работы'] 
                and r.parentTech=='Фиброгипс'
        else r[col] 
        , axis=1
    )

In [187]:
# shall be empty
# if there is no parent then there shall be a `orderRowNo`
fill_row_nos = prods[(prods.parentItem.isnull()) & (prods.orderRowNo.isnull())]    

In [188]:
# fill missing orderRowNo
for i, row in fill_row_nos.iterrows():
    subset = order[(order.spec==row.owner) & (order.tech==row.tech)]
    if len(subset) != 1:
        print(f'Cannot match order row for prod {row.prodNo} {row["item"]}')
        continue
    prods.at[i, 'orderRowNo'] = subset.orderRowNo.values[0]
    prods.at[i, 'spec'] = subset.spec.values[0]

In [189]:
prods[(prods.parentItem.notnull()) & (prods.orderRowNo.notnull())].tech.unique()    # shall be `Фиброгипс` only

array(['Фиброгипс'], dtype=object)

In [190]:
prod_status_codes = {
    'В очереди на производство': 1,
    'В работе': 2,
    'На стеллажах': 3,
    'Отгружен': 4,
    'Завершен': 5,
    'В ремонте': 1
}

In [191]:
prod_status_codes_reverse = {v: k for k, v in prod_status_codes.items() 
                            if k!='В ремонте'}

In [192]:
prods['prodStatusCode'] = prods.prodStatus.map(prod_status_codes)

# Get budget

Which rate is used to arrive at the price an order is sold?

In [193]:
products = order[order.tech.isin([
    'Фиброгипс',
    'Отливка',
    'Протяжка',
    'Отливная тяга резина', 
    'Отливная тяга пластик'
])][['item', 'itemId']].drop_duplicates()

In [194]:
product_ids = "'" + "', '".join(products.itemId) + "'"

In [195]:
price_query = f'''
-- all_prices_4.sql
WITH d AS (
    SELECT 
        CAST(items.[_IDRRef] AS uniqueidentifier) AS itemId
        , items._Description AS Наименование
        , CAST(DATEADD([YEAR], -2000, _Period) AS date) AS Дата
        , ROW_NUMBER()  OVER (
                                PARTITION BY 
--                                    CAST(items._Fld1527 AS int)  -- артикул
                                    items.[_IDRRef]
                                    , price_types._Description      -- ВидЦены
                                ORDER BY 
                                    CAST(DATEADD([YEAR], -2000, _Period) AS date) DESC  -- Дата
--                                ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING
                                )
            AS row_num
        , prices._Fld7568 AS Цена
        , price_types._Description AS ВидЦены
    FROM _InfoRg7564 AS prices
        LEFT JOIN _Reference36 AS price_types ON prices._Fld7565RRef = price_types._IDRRef
        LEFT JOIN _Reference76 AS items ON prices._Fld7566RRef = items._IDRRef
        LEFT JOIN _Reference79 as tech ON items._Fld1533RRef = tech._IDRRef 
        LEFT JOIN _Reference68 AS partners ON items._Fld16820RRef = partners._IDRRef
    WHERE
        price_types._Description IN ('Исполнитель', '{price_type}')
        AND (
            CAST(items._IDRRef AS uniqueidentifier) IN ({product_ids})
            OR CAST(items._Fld1527 AS int) = 200032    -- артикул шаблона
        )
) 
SELECT 
    Наименование
    , itemId
    , Дата
    , ВидЦены
    , Цена
FROM d
WHERE row_num = 1
ORDER BY Наименование, ВидЦены
'''

prices = pd.read_sql(price_query, engine_unf)

In [196]:
specs_list = list(order.specId.unique())
specs_list = [spec for spec in specs_list if pd.notnull(spec)]
specs_list = "', '".join(specs_list)
specs_query = f'''
-- complex_specs.sql
SELECT 
    specs._Description AS spec
    , CAST(specs._IDRRef AS uniqueidentifier) AS specId
    , parent._Description AS owner
    , CAST(parent._IDRRef AS uniqueidentifier) AS ownerId
    , parent_tech._Description AS ownerTech
    , CAST(_LineNo16156 AS int) AS specLineNo
    , CAST(component._IDRRef AS uniqueidentifier) AS componentId
    , component._Description AS component
    , component_tech._Description AS componentTech
    , components_list._Fld36159 AS rate
FROM 
    _Reference16144 AS specs
    LEFT JOIN _Reference76 AS parent ON specs._Fld16369RRef = parent._IDRRef
    LEFT JOIN _Reference79 AS parent_tech ON parent._Fld1533RRef = parent_tech._IDRRef
    LEFT JOIN _Reference16144_VT16155 AS components_list 
            ON components_list._Reference16144_IDRRef  = specs._IDRRef
    LEFT JOIN _Reference76 AS component ON components_list._Fld16157_RRRef = component._IDRRef
    LEFT JOIN _Reference79 AS component_tech ON component._Fld1533RRef = component_tech._IDRRef
WHERE 
    CAST(specs.[_IDRRef] AS uniqueidentifier) IN ('{specs_list}')
ORDER BY 
    CAST(specs.[_Code] AS int)
    , CAST(components_list._LineNo16156 AS int)   -- specLineNo
'''

specs = pd.read_sql(specs_query, engine_unf)

In [197]:
def get_rate(row):
    try:
        if row.tech in ["Фиброгипс", "Протяжка", "Отливка", "Шаблон",
                "Отливная тяга резина", "Отливная тяга пластик"]:   
            rate = prices[
                (prices.itemId==row.itemId) &
                (prices.ВидЦены=="Исполнитель")
            ].Цена.values[0]
            return rate
        
        return specs[(row.itemId==specs.componentId) & (row.specId==specs.specId)].rate.values[0]
    except Exception as e:
        print(e)
        raise e

order['rate'] = order.apply(get_rate, axis=1)

In [198]:
order['budget'] = order.rate * order.qty

In [199]:
assert len(order[(order.budget.isnull()) | (order.budget==0)])==0, 'Not all order row budget calculated.'

# Execution (Job Tickets)

In [200]:
prod_ids = list(prods.prodId.values)
prod_ids = "', '".join(prod_ids)
jobs_query = f'''
-- @job_tickets_tables.sql
-- pulls & castings shops pay monthly
SELECT
    CAST(DATEADD([YEAR], -2000, ticket._Date_Time) AS date) AS jobDate,
    ticket._Number AS job,
    prod_order.[_Number] AS prodOrderNo,
    CAST(prod_order.[_IDRRef] AS uniqueidentifier) AS prodOrderId,
    item._Description AS item,
    CAST(item._IDRRef AS uniqueidentifier) AS itemId,
    owner.[_Description] AS owner,
    CAST(owner._IDRRef AS uniqueidentifier) AS ownerId,
    tech._Description AS tech,
    CAST(tbl._Fld5276RRef AS uniqueidentifier) AS batchId,
    tbl._Fld5271 AS quantity,
    tbl.[_Fld5273] AS rate,
    CAST(tbl._Fld5275 AS float) AS pay
FROM 
    _Document209 ticket 
    LEFT JOIN _Document209_VT5262 tbl ON ticket._IDRRef = tbl._Document209_IDRRef
    LEFT JOIN _Reference76 item ON tbl._Fld5266RRef = item._IDRRef
    LEFT JOIN _Reference79 tech ON tech._IDRRef = item._Fld1533RRef
    LEFT JOIN _Document163 prod_order ON prod_order._IDRRef = ticket._Fld16366RRef
    LEFT JOIN _Document163_VT3562 AS prod_tables 
        ON prod_order._IDRRef = prod_tables._Document163_IDRRef
        AND tbl._LineNo5263 = prod_tables._LineNo3563
    LEFT JOIN _Reference76 owner ON owner.[_IDRRef] = prod_tables._Fld33461RRef 
WHERE 
    CAST(ticket._Posted AS int) = 1
    AND CAST(item._Fld1527 AS int) > 0     -- item_id
    AND CAST(ticket._Fld16366RRef AS uniqueidentifier) IN ('{prod_ids}')
'''
jobs = pd.read_sql(jobs_query, engine_unf)

In [201]:
assert sum(jobs.batchId.isnull()) == 0, 'Job tickets missing batch.'

In [202]:
assert sum(jobs.prodOrderId.isnull()) == 0, 'Production order is missing in Job ticket.'

In [203]:
def get_order_row(prodId):
    prod = prods[prods.prodId==prodId]
    if len(prod) > 0:
        if pd.notnull(prod.orderRowNo.values[0]):
            return prod.orderRowNo.values[0]
        if pd.notnull(prod.orderRowNo_parent.values[0]):
            return prod.orderRowNo_parent.values[0]
        return get_order_row(prod.parentProdId.values[0])

jobs['orderRowNo'] = jobs.prodOrderId.apply(get_order_row)

In [204]:
assert sum(jobs.orderRowNo.isnull())==0, 'Row no not assigned'

In [205]:
def get_completion_date(row):
    if row.tech in ['Модельные работы', 'Формовочные работы'] and row.prodStatus=='Завершен':
        subset = jobs[jobs.prodOrderId==row.prodId]
        if len(subset) > 0:
            return jobs.jobDate.values[0]
    return

prods['completionDate'] = prods.apply(get_completion_date, axis=1)

# Calc diff

In [206]:
jobs_grouped = jobs.groupby(by='orderRowNo').agg('sum', numeric_only=True).pay

In [207]:
order = order.merge(
    jobs_grouped,
    how='left',
    left_on='orderRowNo',
    right_index=True,
    suffixes=('', '_jobs')
)

In [208]:
assert math.isclose(order.pay.sum(), jobs.pay.sum(), rel_tol=0.001), 'Jobs allocation error.'

# Order Row Min Status

In [209]:
row_min_status = prods[['orderRowNo', 'prodStatusCode']].groupby('orderRowNo').agg(min)

In [210]:
order = order.merge(row_min_status, how='left', 
                    left_on='orderRowNo', right_index=True)
order.rename(columns={'prodStatusCode': 'statusCode'}, inplace=True)

In [211]:
order['status'] = order.statusCode.map(prod_status_codes_reverse)

In [212]:
order.to_sql(
    name='orderBudgetRpt',
    con=engine_analytics,
    if_exists='replace',
    index=False
)

-1

# Actual Cost of Work Performed (ACWP)

In [213]:
acwp = jobs[['jobDate', 'pay']].groupby('jobDate').agg('sum')

In [214]:
acwp.rename(columns={'pay': 'acwp'}, inplace=True)

In [215]:
acwp.to_sql(
    name='orderAcwp',
    con=engine_analytics,
    if_exists='replace'
)

-1

# Budgeted Cost of Work Performed (BCWP)

Only production orders directly subordinated to customer order rows count.  

If MFR production order is not in `Completed` status then MFR considered incomplete in whole.  
For completed MFR completion date is the date of directly subordinated job ticket.  

Products can be partially completed based on the number of units billed over total  
number of units in the customer order row.  
For products completion date is not calculated in general. Rather there are  
multiple dates with partial job tickets.

Output:
| calendarDate | BCWS | BCWS cumulative |
|--------------|------|-----------------|

In [216]:
calendar_query = '''
  SELECT 
    CAST(DATEADD([YEAR], -2000, _Fld6647) AS date) date
    --, CAST(_Fld6649RRef AS uniqueidentifier) AS ВидДня
    , CASE
        WHEN CAST(_Fld6649RRef AS uniqueidentifier) IN ('DE79BA8C-92F0-7714-4F63-B787C3B5C81F', '9031DDB1-C61A-DC1C-4A16-97C6C8B0A1BE') THEN 1
        ELSE 0
    END AS isWorkDay
  FROM _InfoRg6645
  WHERE _Fld6648 BETWEEN 2022 AND 2025   -- year
'''
calendar = pd.read_sql(calendar_query, engine_unf)

In [217]:
bcwp_mfr = order[[
    'orderRowNo',
    'tech',
    'spec',
    'item',
    'qty',
    'budget',
    'statusCode'
]]
bcwp_mfr = bcwp_mfr[bcwp_mfr.tech.isin(['Шаблон', 'Модельные работы', 'Формовочные работы'])]

In [218]:
directly_subordinated_prods = prods[
    (prods.orderRowNo.isin(bcwp_mfr.orderRowNo)) 
    & (prods.parentProdId.isnull())
    & (prods.posted==1)
]
direct_grouped = directly_subordinated_prods[['prodNo', 'orderRowNo']]\
                        .groupby('orderRowNo').agg('count')

assert sum(direct_grouped.prodNo!=1) == 0, \
    'More than one production orders directly subordinated to a row in the customer order.'

In [219]:
def get_mfr_bcwp(row):
    if row.tech in ["Модельные работы", "Формовочные работы", "Шаблон"]:
        direct_prods = prods[
            (prods.orderRowNo==row.orderRowNo) 
            & (prods.parentProdId.isnull())
            & (prods.posted==1)        
        ]

        if len(direct_prods) == 0:
            return 

        if direct_prods.prodStatus.values[0] not in ["Завершен", "На стеллажах", "Отгружен"]:     
            return 

        prod_id = direct_prods.prodId.values[0]

        # chk directly subordinated job tickets
        subset = jobs[(jobs.prodOrderId==prod_id)]
        if len(subset) > 0:
            job_completion_date = subset.jobDate.max()
            return job_completion_date
        
        # else look for subordinated prod orders and their job tickets
        sub_prods = prods[
            (prods.parentProdId.isin(direct_prods.prodId)) &
            (prods.posted==1)
        ]
        
        subset = jobs[jobs.prodOrderId.isin(sub_prods.prodId.values)]

        if len(subset) > 0:
            job_completion_date = subset.jobDate.max()
            return job_completion_date
    else:
        return


bcwp_mfr['date'] = bcwp_mfr.apply(get_mfr_bcwp, axis=1)
bcwp_mfr = bcwp_mfr[bcwp_mfr.date.notnull()]

In [220]:
bcwp_mfr.head()

,orderRowNo,tech,spec,item,qty,budget,statusCode,date
1,2,Модельные работы,иДс-6432 СФГ часть 1,МР Сложная/мелкая геометрия,1.0,73594.0,5.0,2022-08-31
4,5,Модельные работы,иДс-6432 СФГ часть 2,МР Сложная/мелкая геометрия,1.0,10420.0,5.0,2022-08-31
7,8,Модельные работы,иДс-6433 СФГ часть 1,МР Сложная/мелкая геометрия,1.0,55595.0,5.0,2022-08-31
8,9,Формовочные работы,иДс-6433 СФГ часть 1,"ФР Тариф 3, без фанеры (площадь более либо рав...",1.0,15443.0,5.0,2022-10-31
10,11,Модельные работы,иДс-6433 СФГ часть 2,МР Сложная/мелкая геометрия,1.0,10819.0,5.0,2022-08-31


In [221]:
bcwp_products = jobs[jobs.tech.isin(["Фиброгипс", "Протяжка"])]
bcwp_products.rename(columns={'jobDate': 'date'}, inplace=True)

C:\Users\kan\AppData\Local\Temp\ipykernel_19740\3766464463.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bcwp_products.rename(columns={'jobDate': 'date'}, inplace=True)


In [222]:
bcwp = pd.concat([bcwp_mfr[['date', 'budget']], bcwp_products[['date', 'quantity', 'rate']]])

In [223]:
bcwp.budget = bcwp.apply(lambda r: r.budget if pd.notnull(r.budget)
                         else r.quantity * r.rate, axis=1)

In [224]:
bcwp = bcwp[['date', 'budget']]
bcwp.rename(columns={'budget': 'bcwp'}, inplace=True)

In [225]:
bcwp = bcwp.groupby('date').agg('sum')

In [228]:
bcwp['bcwp_acc'] = bcwp.bcwp.cumsum()

In [231]:
bcwp.to_sql(
    name='orderBcwp',
    con=engine_analytics,
    if_exists='replace'
)

-1

# Output

In [41]:
f = graphviz.Digraph(filename="output.gv")
f.attr(rankdir='LR')  
for _, r in prods.sort_values(by='orderRowNo', ascending=False).iterrows():
    match r.tech:
        case 'Фиброгипс':
            f.attr('node', shape='box')
        case 'Модельные работы':
            f.attr('node', shape='cds')
        case 'Формовочные работы':
            f.attr('node', shape='octagon')
        case 'Шаблон':
            f.attr('node', shape='trapezium')
        case 'Протяжка':
            f.attr('node', shape='house')
        case _:
            f.attr('node', shape='')

    if pd.notnull(r.orderRowNo):
        f.attr('node', style='filled', fillcolor='lightblue')
    else:
        f.attr('node', style='', fillcolor='')
    label = (
            (f'c.{r.orderRowNo:.0f}\n' if pd.notnull(r.orderRowNo) else '') +
            f'{r.prodNo}\n' +
            (f'{r.spec}' if pd.notnull(r.spec) else '---') +
            f'\n{r["item"]}'
    )
    f.node(r.prodNo, label)
for _, r in prods[prods.parentProdNo.notnull()].iterrows():
    if (
            r.parentTech == 'Фиброгипс' 
            and r.tech in ['Модельные работы', 'Формовочные работы']
    ): continue
    f.edge(r.parentProdNo, r.prodNo)

f.render()

'output.gv.pdf'